In [4]:
import time
import serial

ImportError: No module named 'serial'

In [87]:
ser = serial.Serial(
    port='/dev/tty.usbserial-A400936U',
    baudrate=9600,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    bytesize=serial.EIGHTBITS
)

In [93]:
ser.isOpen()

False

In [28]:
D = []
for i in range(0,16,1):
    D.append(v_str[i:i+1])

# D[0]: Start Word = 2


# D[1]: Reading polarity for the display
# 0 = Both upper & lower display value are "+"
# 1 = Upper "-", Lower "+"
# 2 = Upper "+", Lower "-"
# 3 = Both upper & lower display value are "-"


# D[2]: Annunciator for Lower Display
# 0 = No Symbol
# 1 = ℃
# 2 = ℉


# D[3:5]: Annunciator for Upper Display
# 01 = ℃
# 02 = ℉
# 07 = mg/L
# 06 = % O2


# D[5]: Decimal Point (DP) for Lower display
# 0 = No DP
# 1 = 1 DP
# 2 = 2 DP
# 3 = 3 DP


# D[6]: Decimal Point (DP) for Upper display
# 0 = No DP
# 1 = 1 DP
# 2 = 2 DP
# 3 = 3 DP


# D[7:11]: Lower Display reading, 4 digits, D[7] = MSD, D[10] = LSD

# D[11:15]: Upper Display reading, 4 digits, D[11] = MSD, D[14] = LSD 
  # [D4, D3, D2, D1]

# D[15]: End Word = 0D

[b'\x02',
 b'0',
 b'1',
 b'0',
 b'6',
 b'1',
 b'1',
 b'0',
 b'2',
 b'8',
 b'0',
 b'0',
 b'2',
 b'1',
 b'0',
 b'\r']

In [88]:
def b2i(b_char):
    if b_char == b'0':
        return 0
    elif b_char == b'1':
        return 1
    elif b_char == b'2':
        return 2
    elif b_char == b'3':
        return 3
    elif b_char == b'4':
        return 4
    elif b_char == b'5':
        return 5
    elif b_char == b'6':
        return 6
    elif b_char == b'7':
        return 7
    elif b_char == b'8':
        return 8
    elif b_char == b'9':
        return 9

In [89]:
def parse_D(D):
    reading = {}
    
    # Lower Display reading
    LD = 0
    LD = LD + b2i(D[7]) * 10**3
    LD = LD + b2i(D[8]) * 10**2
    LD = LD + b2i(D[9]) * 10
    LD = LD + b2i(D[10])
    LD = LD / 10**b2i(D[5])
    reading['LD'] = LD
    
    # Lower Display Annunciator
    if D[2] == b'0':
        LDA = 'NO SYMBOL'
    elif D[2] == b'1':
        LDA = '℃'
    elif D[2] == b'2':
        LDA = '℉'
    
    reading['LDA'] = LDA
    
    # Upper Display reading
    UD = 0
    UD = UD + b2i(D[11]) * 10**3
    UD = UD + b2i(D[12]) * 10**2
    UD = UD + b2i(D[13]) * 10
    UD = UD + b2i(D[14])
    UD = UD / 10**b2i(D[6])
    reading['UD'] = UD
    
    # Upper Display Annunciator
    if D[4] == b'0':
        UDA = 'NO SYMBOL'
    elif D[4] == b'1':
        UDA = '℃'
    elif D[4] == b'2':
        UDA = '℉'
    elif D[4] == b'7':
        UDA = 'mg/L'
    elif D[4] == b'6':
        UDA = '%O2'
    
    reading['UDA'] = UDA
    
    # Reading polarity for the display
    if D[1] == b'0':
        pass
    elif D[1] == b'1':
        reading['UD'] = -1.0 * reading['UD']
    elif D[1] == b'2':
        reading['LD'] = -1.0 * reading['LD']
    elif D[1] == b'3':
        reading['UD'] = -1.0 * reading['UD']
        reading['LD'] = -1.0 * reading['LD']
    
    return reading

In [91]:
while 1:
    ser.reset_input_buffer()
    v_str = ser.read(16)
    D = []
    for i in range(0,16,1):
        D.append(v_str[i:i+1])
    reading = parse_D(D)
    print(reading)
    time.sleep(1)

{'LD': 27.9, 'UD': 20.8, 'UDA': '%O2', 'LDA': '℃'}
{'LD': 27.9, 'UD': 20.8, 'UDA': '%O2', 'LDA': '℃'}
{'LD': 28.0, 'UD': 20.7, 'UDA': '%O2', 'LDA': '℃'}
{'LD': 28.0, 'UD': 20.7, 'UDA': '%O2', 'LDA': '℃'}
{'LD': 27.9, 'UD': 20.8, 'UDA': '%O2', 'LDA': '℃'}
{'LD': 28.0, 'UD': 20.7, 'UDA': '%O2', 'LDA': '℃'}
{'LD': 28.0, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℃'}
{'LD': 27.9, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℃'}
{'LD': 27.9, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℃'}
{'LD': 27.8, 'UD': 7.9, 'UDA': 'mg/L', 'LDA': '℃'}
{'LD': 27.9, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℃'}
{'LD': 82.2, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℉'}
{'LD': 82.2, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℉'}
{'LD': 82.0, 'UD': 7.9, 'UDA': 'mg/L', 'LDA': '℉'}
{'LD': 82.2, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℉'}
{'LD': 82.2, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℉'}
{'LD': 82.4, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℉'}
{'LD': 82.4, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℉'}
{'LD': 82.4, 'UD': 7.8, 'UDA': 'mg/L', 'LDA': '℉'}
{'LD': 82.2, 'UD': 7.8, 'UDA': 

KeyboardInterrupt: 

In [39]:
ser.close()
ser.isOpen()

False

In [28]:
ser.closed

False

In [83]:
ser.open()

In [92]:
ser.close()